Comparing bonus unemployment, payroll tax cuts, and universal payments
=============================================================

This uses the 2018 Current Population Survey March Supplement, Tax-Calculator, and the Supplemental Poverty Measure to estimate the effects of the Federal Pandemic Unemployment Compensation (extra 600 dollars per week), a budget-neutral payroll tax cuts, and a budget-neutral universal payment.

In [1]:
import numpy as np
import pandas as pd
import microdf as mdf
import plotly.express as px
import plotly

spmu = pd.read_feather('data/spmu.feather')
person = pd.read_feather('data/person.feather')

# All potential reforms.
CHG_COLS = ['fpuc_net', 'fpuc_ubi', 'fpuc_adult_ubi', 'fpuc_fica_cut',
            'fpuc2_net', 'fpuc2_ubi', 'fpuc2_adult_ubi', 'fpuc2_fica_cut']

## Poverty analysis

In [2]:
def pov(reform, year, age_group='All', race='All'):
    """ Calculate the poverty rate under the specified reform for the
        specified population.
        Note: All arguments refer to the poverty population, not the reform.
    
    Args:
        reform: One of CHG_COLS. If None, provides the baseline rate.
        year: Year of the data (year before CPS survey year).
        age_group: Age group, either
            - 'Children' (under 18)
            - 'Adults' (18 or over)
            - 'All'
        race: Race code to filter to. Defaults to 'All'.
        
    Returns:
        2018 SPM poverty rate.
    """
    # Select the relevant poverty column for the reform.
    if reform == 'baseline':
        poverty_col = 'spmpoor'
    else:
        poverty_col = 'spmpoor_' + reform
    # Filter by year.
    target_persons = person[person.FLPDYR == year]
    # Filter by age group.
    if age_group == 'Children':
        target_persons = target_persons[target_persons.age < 18]
    elif age_group == 'Adults':
        target_persons = target_persons[target_persons.age >= 18]
    # Filter by race.
    if race != 'All':
        target_persons = target_persons[target_persons.race == race]
    # Return poverty rate (weighted average of poverty flag).
    return mdf.weighted_mean(target_persons, poverty_col, 'asecwt')


def pov_row(row):
    """ Calculate poverty based on parameters specified in the row.
    
    Args:
        row: pandas Series.
        
    Returns:
        2018 SPM poverty rate.
    """
    return pov(row.reform, row.year, row.age_group, row.race)

In [3]:
pov_rates = mdf.cartesian_product({'reform': ['baseline'] + CHG_COLS,
                                   'year': person.FLPDYR.unique(),
                                   'age_group': ['All', 'Children', 'Adults'],
                                   'race': ['All', 200]})  # 200 means Black.
pov_rates['pov'] = 100 * pov_rates.apply(pov_row, axis=1)

Top-level poverty rates for latest year.

In [4]:
pov_rates[(pov_rates.age_group == 'All') & (pov_rates.race == 'All') &
          (pov_rates.year == 2018)]

,reform,year,age_group,race,pov
54,baseline,2018,All,All,12.735766
114,fpuc_net,2018,All,All,11.988565
174,fpuc_ubi,2018,All,All,12.176865
234,fpuc_adult_ubi,2018,All,All,12.235833
294,fpuc_fica_cut,2018,All,All,12.576446
354,fpuc2_net,2018,All,All,11.794734
414,fpuc2_ubi,2018,All,All,11.664399
474,fpuc2_adult_ubi,2018,All,All,11.685191
534,fpuc2_fica_cut,2018,All,All,11.890402


Top-level child poverty rates.

In [5]:
pov_rates[(pov_rates.age_group == 'Children') & (pov_rates.race == 'All') &
          (pov_rates.year == 2018)]

,reform,year,age_group,race,pov
56,baseline,2018,Children,All,13.657287
116,fpuc_net,2018,Children,All,12.658798
176,fpuc_ubi,2018,Children,All,12.880944
236,fpuc_adult_ubi,2018,Children,All,13.171264
296,fpuc_fica_cut,2018,Children,All,13.433809
356,fpuc2_net,2018,Children,All,12.388159
416,fpuc2_ubi,2018,Children,All,12.220106
476,fpuc2_adult_ubi,2018,Children,All,12.378253
536,fpuc2_fica_cut,2018,Children,All,12.509854


### Poverty gap and inequality

Calculate these for all people and SPM units, without breaking out by age or race.

In [6]:
def pov_gap_b(reform, year):
    if reform == 'baseline':
        resource_col = 'spmtotres'
    else:
        resource_col = 'spmtotres_' + reform
    tmp = spmu[spmu.FLPDYR == year]
    pov_gap = np.maximum(tmp.spmthresh - tmp[resource_col], 0)
    return (pov_gap * tmp.spmwt).sum() / 1e9

def pov_gap_row(row):
    return pov_gap_b(row.reform, row.year)

In [7]:
pov_gap_ineq = pov_rates[['reform', 'year']].drop_duplicates()
pov_gap_ineq['pov_gap_b'] = pov_gap_ineq.apply(pov_gap_row, axis=1)

### Inequality

By individual based on their percentage of SPM resources.

In [8]:
def gini(reform, year):
    if reform == 'baseline':
        resource_col = 'spmtotres'
    else:
        resource_col = 'spmtotres_' + reform
    tmp = person[person.FLPDYR == year]
    return mdf.gini(tmp[resource_col], tmp.asecwt)

def gini_row(row):
    return gini(row.reform, row.year)

In [9]:
pov_gap_ineq['gini'] = pov_gap_ineq.apply(gini_row, axis=1)
pov_gap_ineq[pov_gap_ineq.year == 2018]

,reform,year,pov_gap_b,gini
54,baseline,2018,169.993787,0.431619
114,fpuc_net,2018,162.096358,0.428555
174,fpuc_ubi,2018,162.778373,0.428992
234,fpuc_adult_ubi,2018,162.881292,0.429285
294,fpuc_fica_cut,2018,168.802118,0.431637
354,fpuc2_net,2018,160.711410,0.427140
414,fpuc2_ubi,2018,157.749994,0.426917
474,fpuc2_adult_ubi,2018,157.791357,0.427099
534,fpuc2_fica_cut,2018,161.398876,0.428578
